### Library Imports

In [ ]:
#libraries:
import pandas as pd
import seaborn as sns
import numpy as np
import requests
import xml.etree.ElementTree as ET
import xmltodict
import matplotlib.pyplot as plt
import json

#Dependencies:
from config import api_key, google_api_key

GoogleBooks/ GoodReads data cleaning, visualisation

In [ ]:
#define global variables:
books=[]
isbns=[]
results=[]

First 10 results from google API for fiction books

In [ ]:
#define categories for books:
#categories=['fiction','nonfiction']      #'Fantasy','Thriller',"adventure","Horror","Pyschology","History","romance"

category='fiction'

#parameters
params={'startIndex':480,
    'key':google_api_key,
       'maxResults':40}

url= f'https://www.googleapis.com/books/v1/volumes?q=subject:{category}'

#get request
response= requests.get(url, params).json()

result=response['items']

results.append(result)

In [ ]:
category='nonfiction'
#parameters
params={'startIndex':160,
    'key':google_api_key,
       'maxResults':40}

url= f'https://www.googleapis.com/books/v1/volumes?q=subject:{category}'

#get request
response= requests.get(url, params).json()
#result=response['items'] if 'items' in response.keys() else " "

#results.append(result)
result=response['items']

results.append(result)

In [ ]:
#action, adventure, crime, thriller, horror, mystery, religion, adult, science, philosophy,literature,drama
category='comedy'
#parameters
params={'startIndex':120,
    'key':google_api_key,
       'maxResults':40}

url= f'https://www.googleapis.com/books/v1/volumes?q=subject:{category}'

#get request
response= requests.get(url, params).json()
#result=response['items'] if 'items' in response.keys() else " "

#results.append(result)
result=response['items']

results.append(result)

In [ ]:
len(results[1])

In [ ]:
len(results)

In [ ]:
import itertools
import time
books=[]

In [ ]:

#explore response from google API
for i in range(60,70):      #range(len(results)):    
   
    book={}
    for item in results[i]:
        try:
            book={
                'image_url':item['volumeInfo']['imageLinks']['thumbnail'] if 'imageLinks' in item['volumeInfo'].keys() else " ",
                'id_book': item['id'],
                'title':item['volumeInfo']['title'] if 'title' in item['volumeInfo'].keys() else " ",
                'category/genre':item['volumeInfo']['categories'][0] if 'categories' in item['volumeInfo'].keys() else " ",
                'authors':item['volumeInfo']['authors'] if 'authors' in item['volumeInfo'].keys() else " ",
                'description': item['volumeInfo']['description'] if 'description' in item['volumeInfo'].keys() else " ",
                'isbn':item['volumeInfo']['industryIdentifiers'][0]['identifier'] if 'industryIdentifiers' in item['volumeInfo'].keys() else " ",
                'language':item['volumeInfo']['language'] if 'language' in item['volumeInfo'].keys() else " ",
                'published_date':item['volumeInfo']['publishedDate'] if 'published_date' in item['volumeInfo'].keys() else " ",
                'publisher': item['volumeInfo']['publisher'] if 'publisher' in item['volumeInfo'].keys() else " "     
            }

        except:
            book = {'id_book': 'not found'}

        #isbns.append(book['isbn'])
        #print(book['isbn'])
       
        #Good books review counts for isbn_string
        url_gr='https://www.goodreads.com/book/review_counts.json'

        parameters_gr={"key":api_key,
            "format":'json',
            "isbns":book['isbn']
        }
        
        try:
            #if ISBN not found in goodreads
            if requests.get(url_gr, params=parameters_gr).text!='No books match those ISBNs.':

                #time delay
                #time.sleep(2)

                results_gr=requests.get(url_gr, params=parameters_gr).json()


                id=results_gr['books'][0]['id']
                # use goodreads book ID to get ratings distribution

                #time delay
                #time.sleep(1)

                #Set URL
                url_gr2='https://www.goodreads.com/book/show/'

                #set parameters
                parameters_gr2={
                    "key":api_key,
                    "format":'xml',
                    "id":id
                }
                #get requests
                results_gr2=requests.get(url_gr2, params=parameters_gr2)

                #parse XML to string, define root
                root_gr= ET.fromstring(results_gr2.text)

                rating_dist=root_gr.findall('.//rating_dist')[0].text

                #print(rating_dist)

                rating={"5-stars":rating_dist.split("|")[0].split(':')[1],
                    "4-stars":rating_dist.split("|")[1].split(':')[1],
                    "3-stars":rating_dist.split("|")[2].split(':')[1],
                    "2-stars":rating_dist.split("|")[3].split(':')[1],
                    "1-star":rating_dist.split("|")[4].split(':')[1]
                       }

                reviews={"ratings_count":results_gr['books'][0]['work_ratings_count'],
                        "reviews_count":results_gr['books'][0]['work_reviews_count'],
                        'text_reviews_count':results_gr['books'][0]['work_text_reviews_count'],
                        'average_rating':results_gr['books'][0]['average_rating'],
                        'rating_dist':rating
                         }
                
                shelves_gr=root_gr.findall('.//popular_shelves/shelf')
                shelves=[]
                
                for i in range(len(shelves_gr)):
                    shelves.append(shelves_gr[i].attrib)
                
                #Apply filter for sub_categories based on shelves
                count= list(map(lambda x: int(x['count']), list(filter(lambda entry: [entry for (k,v) in entry.items() if v=='fiction' or v=='adventure' or v=='survival'or v=='childrens'
                           or v=='contemporary-fiction' or v=='action' or v=='nature' or v=='drama' or v=='juvenile-fiction'
                          or v=='action-adventure'or v=='young-adult-fiction'or v=='wilderness'or v=='fantasy'or v=='horror'
                        or v=='adult' or v=='religion'or v=='historical-fiction' or v=='history'or v=='mystery'or v=='contemporary'
                        or v=='adult-fiction'or v=='literature' or v=='general-fiction'or v=='thriller'or v=='philosophy'or v=='science'
                          ], shelves))))

                sub_category=[item['name'] for item in shelves if int(item['count'])== max(count)][0]
                
                num_pages=root_gr.findall('.//book/num_pages')[0].text

                # append reviews to book
                book['reviews']=reviews
                book['sub_category']=sub_category
                book['num_pages']=num_pages

                #append books
                books.append(book)
                print('ok')
            
        except:
            ValueError
        


In [ ]:
len(books)

In [ ]:
9780399591785
 

In [ ]:
books_list

In [ ]:
#convert to books dataframe
books_df=pd.DataFrame(books)
books_df.head()

In [ ]:
#expand reviews columns to separate fields
books_df['ratings_count']=books_df['reviews'].apply(lambda x:x['ratings_count'])
books_df['reviews_count']=books_df['reviews'].apply(lambda x:x['reviews_count'])
books_df['text_reviews_count']=books_df['reviews'].apply(lambda x:x['text_reviews_count'])
books_df['average_rating']=books_df['reviews'].apply(lambda x:x['average_rating'])

#final dataframe
books_df.head()


In [ ]:
#books_df.to_csv('books_df_all.csv',index=False)

In [ ]:
books_df.info()

# to SQL

In [31]:
from sqlalchemy import create_engine
connection_string = 'postgres:postgres@localhost:5432/BookShare'
engine = create_engine(f'postgresql://{connection_string}')

In [ ]:
engine.table_names()

In [ ]:
books_sql=books_df.drop(['reviews'], axis=1).dropna()
books_sql.info()

In [ ]:
books_sql.to_csv('books_sql.csv',index=False)

In [7]:
import pandas as pd
books_sql=pd.read_csv('books_sql.csv')
books_sql.head()

,authors,category/genre,description,id_book,image_url,isbn,language,num_pages,published_date,publisher,sub_category,title,ratings_count,reviews_count,text_reviews_count,average_rating
0,['Stephen King'],Fiction,Returns to the rich landscape of Mid-World in ...,Nvkwjw2LToEC,http://books.google.com/books/content?id=Nvkwj...,9781451658903,en,309,,Simon and Schuster,fantasy,The Wind Through the Keyhole,62949,130610,5030,4.15
1,['Jill McCorkle'],Fiction,Jo Spencer's conventional life is disrupted wh...,ouTaAnG0ipwC,http://books.google.com/books/content?id=ouTaA...,9781565120013,en,400,,Algonquin Books,fiction,The Cheer Leader,473,884,36,3.71
2,['Jules Verne'],Fiction,Jules Verne: Die Kinder des Kapitän Grant. All...,ekDTBgAAQBAJ,http://books.google.com/books/content?id=ekDTB...,9783843030885,de,538,,BoD – Books on Demand,adventure,Die Kinder des Kapitän Grant,9954,17743,224,4.08
3,['Cathy Gillen Thacker'],Fiction,"With a busy career and three kids to feed, sin...",BtMwEKkGwowC,http://books.google.com/books/content?id=BtMwE...,9781426841453,en,224,,Harlequin,hoopla,A Baby for Mommy,70,177,9,3.51
4,['Edgar Wallace'],Fiction,Employing his unique style of innocent and end...,A4KYDQAAQBAJ,http://books.google.com/books/content?id=A4KYD...,9780755122684,en,190,,House of Stratus,fiction,Sanders,27,77,2,3.96


In [8]:
books_sql_copy=books_sql.drop_duplicates(subset=['id_book'],keep='first')
books_sql_copy.head()


,authors,category/genre,description,id_book,image_url,isbn,language,num_pages,published_date,publisher,sub_category,title,ratings_count,reviews_count,text_reviews_count,average_rating
0,['Stephen King'],Fiction,Returns to the rich landscape of Mid-World in ...,Nvkwjw2LToEC,http://books.google.com/books/content?id=Nvkwj...,9781451658903,en,309,,Simon and Schuster,fantasy,The Wind Through the Keyhole,62949,130610,5030,4.15
1,['Jill McCorkle'],Fiction,Jo Spencer's conventional life is disrupted wh...,ouTaAnG0ipwC,http://books.google.com/books/content?id=ouTaA...,9781565120013,en,400,,Algonquin Books,fiction,The Cheer Leader,473,884,36,3.71
2,['Jules Verne'],Fiction,Jules Verne: Die Kinder des Kapitän Grant. All...,ekDTBgAAQBAJ,http://books.google.com/books/content?id=ekDTB...,9783843030885,de,538,,BoD – Books on Demand,adventure,Die Kinder des Kapitän Grant,9954,17743,224,4.08
3,['Cathy Gillen Thacker'],Fiction,"With a busy career and three kids to feed, sin...",BtMwEKkGwowC,http://books.google.com/books/content?id=BtMwE...,9781426841453,en,224,,Harlequin,hoopla,A Baby for Mommy,70,177,9,3.51
4,['Edgar Wallace'],Fiction,Employing his unique style of innocent and end...,A4KYDQAAQBAJ,http://books.google.com/books/content?id=A4KYD...,9780755122684,en,190,,House of Stratus,fiction,Sanders,27,77,2,3.96


In [9]:
books_sql_copy.to_sql(name='library', con=engine, if_exists='replace', index=False)

In [2]:
# Import Dependencies
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine
from sqlalchemy import func

In [32]:
# Reflect Database into ORM classes
Base = automap_base()
Base.prepare(engine, reflect=True)
Base.classes.keys()

['book', 'owner', 'library']

In [83]:
lib= Base.classes.library
session= Session(engine)


In [108]:
books_db=session.query(lib.id_book,lib.title,lib.authors, lib.average_rating).filter(lib.authors.like('%William%')).all()
books_db

[('PS96DQAAQBAJ', 'The Final Day', "['William R. Forstchen']", 4.1),
 ('qvhSPgAACAAJ', 'Renegades', "['William W. Johnstone']", 4.33),
 ('YvJqswEACAAJ',
  'The Killing of Uncle Sam',
  "['Rodney Howard-Browne', 'Paul L. Williams']",
  4.35),
 ('wlFJnjq3ufEC',
  'Antony and Cleopatra',
  "['William Shakespeare', 'David Bevington']",
  3.69),
 ('OjYdDgAAQBAJ',
  'National Geographic Pocket Guide to the Weather of North America',
  "['Jack Williams']",
  4.29),
 ('87fDk-IMFG0C', 'Troilus and Cressida', "['William Shakespeare']", 3.37),
 ('F3_rAgAAQBAJ', 'Twelfth Night', "['William Shakespeare']", 3.98),
 ('kti7vZn1DH0C', 'Macbeth', "['William Shakespeare']", 3.98),
 ('S0TjLV8EvBIC', 'The Miracle Worker', "['William Gibson']", 4.22),
 ('2vF7f03oBcwC',
  'Romeo y Julieta',
  "['William Shakespeare', 'Mariel Ortolano']",
  3.74),
 ('brHZvvgElL4C', 'Picnic', "['William Inge']", 3.83),
 ('WwYCaTrBYFAC',
  'The Glass Menagerie',
  "['Tennessee Williams', 'Tony Kushner']",
  3.7),
 ('qOVvDwAAQBA

In [92]:
books=[]
for book in books_db:
    dict_book = {
        "id" :book[0],
        "title" :book[1],
        "author": book[2],
        "rating":book[3]
    }
    books.append(dict_book)
books

[{'id': 'qaKkenvL29UC',
  'title': 'Harry Potter and the Half-Blood Prince',
  'author': "['J.K. Rowling']",
  'rating': 4.57},
 {'id': '2zgRDXFWkm8C',
  'title': 'Harry Potter y la piedra filosofal',
  'author': "['J.K. Rowling']",
  'rating': 4.47}]

In [86]:
session.close()

In [84]:
books_db=session.query(lib.id_book,lib.title,lib.authors, lib.average_rating).first()

In [85]:
books_db

('Nvkwjw2LToEC', 'The Wind Through the Keyhole', "['Stephen King']", 4.15)

# Adding dummy user data to SQL DATABASE


In [61]:
user_df= pd.DataFrame(columns=['user_id','id_book','title','authors','rating'])

In [62]:
columns=['user_id','id_book','title','authors','review','rating']

In [63]:
user_data=[(69,'Nvkwjw2LToEC', 'The Wind Through the Keyhole', "['Stephen King']","good", 4.15),
          (85,'ouTaAnG0ipwC', 'The Cheer Leader', "['Jill McCorkle']","moderate", 3.71),
           (29,'ekDTBgAAQBAJ', 'Die Kinder des Kapitän Grant', "['Jules Verne']", "good",4.08),
           (89,'YkZdNbGWQH4C', 'Stone Bleeding', "['Russell Mardell']","moderate", 4.0),
            (54,'pLNzDwAAQBAJ', 'Lock Every Door', "['Riley Sager']", "moderate",3.95),
           (9,'gGtjHYVu6cgC', 'Ordinary Beauty', "['Laura Wiess']","good", 4.19),
           (69,'2mgnEzzaJrIC', 'The Joy Luck Club', "['Amy Tan']","moderate",3.92)
          (85,'Nvkwjw2LToEC', 'The Wind Through the Keyhole', "['Stephen King']","good" 4.15),
           (15,'S3K3UK09ZuoC', 'House Rules', "['Jodi Picoult']","good", 4.03),
           (29,'y3gghM_M8U0C', 'Mask Market', "['Andrew H. Vachss']", "moderate",3.81),
           (29,'YkZdNbGWQH4C', 'Stone Bleeding', "['Russell Mardell']", 4.0)
          
          ]

SyntaxError: invalid syntax (<ipython-input-63-e7fda5f4c63e>, line 8)

In [64]:
import numpy as np
np.random.randint(low=1, high=50, size=20)

array([40, 32, 28, 47,  4,  5,  1, 37, 33, 39, 30, 22, 38, 27, 29, 20,  9,
       46,  8, 41])

In [73]:
user_df['user_id']=np.random.randint(low=1, high=50, size=30)

In [66]:
user_df

,user_id,id_book,title,authors,rating
0,49,NaN,NaN,NaN,NaN
1,45,NaN,NaN,NaN,NaN
2,17,NaN,NaN,NaN,NaN
3,19,NaN,NaN,NaN,NaN
4,6,NaN,NaN,NaN,NaN
5,31,NaN,NaN,NaN,NaN
6,4,NaN,NaN,NaN,NaN
7,23,NaN,NaN,NaN,NaN
8,31,NaN,NaN,NaN,NaN
9,6,NaN,NaN,NaN,NaN


In [128]:
user_df=pd.DataFrame(books_db,columns=['id_book','title','authors','rating'])

In [129]:
user_df

,id_book,title,authors,rating
0,PS96DQAAQBAJ,The Final Day,['William R. Forstchen'],4.10
1,qvhSPgAACAAJ,Renegades,['William W. Johnstone'],4.33
2,YvJqswEACAAJ,The Killing of Uncle Sam,"['Rodney Howard-Browne', 'Paul L. Williams']",4.35
3,wlFJnjq3ufEC,Antony and Cleopatra,"['William Shakespeare', 'David Bevington']",3.69
4,OjYdDgAAQBAJ,National Geographic Pocket Guide to the Weathe...,['Jack Williams'],4.29
5,87fDk-IMFG0C,Troilus and Cressida,['William Shakespeare'],3.37
6,F3_rAgAAQBAJ,Twelfth Night,['William Shakespeare'],3.98
7,kti7vZn1DH0C,Macbeth,['William Shakespeare'],3.98
8,S0TjLV8EvBIC,The Miracle Worker,['William Gibson'],4.22
9,2vF7f03oBcwC,Romeo y Julieta,"['William Shakespeare', 'Mariel Ortolano']",3.74


In [109]:
round(1.35)

1

In [110]:
round(1.65)

2

In [113]:
user_df_2=[]
user_df_2['rating']=round(int(user_df['rating']))

TypeError: cannot convert the series to <class 'int'>

In [126]:
user_df['rating']=user_df['rating'].apply(lambda x:float(x)).apply(lambda x:round(x,1))

In [127]:
user_df['rating'].head()

0    4.1
1    4.3
2    4.3
3    3.7
4    4.3
Name: rating, dtype: float64

In [ ]:
round